## DSCI 524 - Collaborative Software Development

### Lecture 5: Introductions to Continuous Integration (CI) & GitHub Actions

#### 2020-03-09

## Lecture 5 learning objectives:
By the end of this lecture, students should be able to:
- [Define continuous integration testing](#Continuous-Integration-(CI))
- [Explain why continuous integration testing is superior to manually running tests](#Why-use-CI-+-automated-testing)
- [Define the following key concepts that underlie GitHub Actions:](#GitHub-actions)
    - Actions
    - Workflow
    - Event
    - Runner
    - Job
    - Step
- Store and use GitHub Actions credentials safely via GitHub Secrets
- Use GitHub Actions to set-up automated running of tests by `pytest` upon push to the master branch
- Use GitHub Actions to set-up automated running of tests by `testthat` upon push to the master branch

## Continuous Integration (CI)

Defined as the practice of **frequently** integrating code (*e.g.*, several times a day) changes from contributors to a shared repository. Often the submission of code to the shared repository is combined with automated testing (and other things, such as style checking) to increase code dependability and quality.

### Why use CI + automated testing

- detects errors sooner
- reduces the amount of code to be examined when debugging
- facilitates merging
- ensures new code additions do not introduce errors

## GitHub actions

A tool and service for automating software development tasks, located in the same place where you already store your code.

### Key concepts:
**Actions:** Individual tasks you want to perform.

**Workflow:** A collection of actions (specified together in one file).

**Event:** Something that triggers the running of a workflow.

**Runner**: A machine that can run the Github Action(s).

**Job**: A set of steps executed on the same runner.

**Step**: A set of commands or actions which a job executes.

### GitHub Actions workflow file:

A `YAML` file that lives in the `.github/workflows` directory or your repository which speciies your workflow. 

```
# This is a basic workflow to help you get started with Actions

name: CI

# Controls when the action will run. Triggers the workflow on push or pull request 
# events but only for the master branch
on:
  push:
    branches: [ master ]
  pull_request:
    branches: [ master ]

# A workflow run is made up of one or more jobs that can run sequentially or in parallel
jobs:
  # This workflow contains a single job called "build"
  build:
    # The type of runner that the job will run on
    runs-on: ubuntu-latest

    # Steps represent a sequence of tasks that will be executed as part of the job
    steps:
    # Checks-out your repository under $GITHUB_WORKSPACE, so your job can access it
    - uses: actions/checkout@v2

    # Runs a single command using the runners shell
    - name: Run a one-line script
      run: echo Hello, world!

    # Runs a set of commands using the runners shell
    - name: Run a multi-line script
      run: |
        echo Add other actions to build,
        echo test, and deploy your project.
```

The file above has:
- one job
- 3 steps
- the type of runner is ubuntu
- the first step uses an action, and the following two steps run commands

### Commands vs actions

Steps can consist commands or actions. Let's spend some time to discuss what each of these are and how they differ.

### Commands

Steps that use commands look like the one shown below. They consist of a `name` and a `run` parameter. The commands listed after `run` are run in the runner's shell:

```
- name: Run a one-line script
      run: echo Hello, world!
```

As shown in the file above, we can run multiple commands in a step using the `|` character:
```
- name: Run a multi-line script
      run: |
        echo Add other actions to build,
        echo test, and deploy your project.
```

### Actions

Steps that use actions look like the one shown below. They always have a `uses` parameter, and often also have `name` and `with` parameters. The `uses` parameter specifies which action to use, and the `with` parameters provide arguments to those actions.

```
- name: checkout
      uses: actions/checkout@master
      with:
        ref: master
        fetch-depth: '0'
```

Actions commonly perform one task in a workflow. They are typically run inside Docker containers on the runner machine. Such actions are defined by:
- a Dockerfile
- a script to run inside the Docker container

For example, for the action above see its:
- [Dockerfile]()
- [endpoint.sh script]()
- [GitHub repo](https://github.com/actions/setup-python)

Read the docs here to learn how to build your own Docker container action: <https://help.github.com/en/actions/building-actions/creating-a-docker-container-action>

> Note: actions can also be built using javascript, for more info see [here](https://help.github.com/en/actions/building-actions/creating-a-javascript-action).

### Exercise: Getting to know GitHub Actions

We are going to each create our own GitHub Actions workflow. 

#### Steps:

1. Create a new public **GitHub.com** repository.

2. Click on the "Actions" tab

3. Click on the first "Set up this workflow" button

    <img src="img/gh-actions-setup.png" width=600>

4. Click on the two green commit buttons to add this workflow file

5. Go to the "Actions" tab and look at the build logs by following these instructions:

    Click on the message associated with the event that created the action:
    
    <img src="img/check-logs1.png" width=600>
    
    Click on the build link:
    
    <img src="img/check-logs2.png" width=600>
    
    Click on the arrow inside the build logs to expand a section and see the output of the action:
    
    <img src="img/check-logs3.png" width=600>


**When you are done step #5 indicate so on the [sli.do](https://www.sli.do) poll (`#524-L05`).**

## Case study: our Python package `build.yml` workflow:

- Let's break down our [`build.yml`](https://github.com/UBC-MDS/cookiecutter-ubc-mds/blob/master/%7B%7Bcookiecutter.project_slug%7D%7D/.github/workflows/build.yml) workflow file to start to better understand a real use case of GitHub Actions.

```
name: build

on: [push, pull_request]

jobs:
  build:
    runs-on: ubuntu-latest
    strategy:
      matrix:
        python-version: [3.7, 3.8]
    steps:
    - uses: actions/checkout@v2
    - name: Set up Python ${{ matrix.python-version }}
      uses: actions/setup-python@v1
      with:
        python-version: ${{ matrix.python-version }}
    - name: Install dependencies
      env:
        POETRY_VIRTUALENVS_CREATE: false
      run: |
        pip install poetry
        poetry install
    - name: Check style
      run: poetry run flake8 --exclude=docs*
    - name: Test with pytest
      run: |
        poetry run pytest --cov=./ --cov-report=xml
    - name: Upload coverage to Codecov  
      uses: codecov/codecov-action@v1
      with:
        token: ${{ secrets.CODECOV_TOKEN }}
        file: ./coverage.xml
        flags: unittests
        name: codecov-umbrella
        yml: ./codecov.yml 
        fail_ci_if_error: true
```

### Exercise: Orientating ourselves with the `build.yml` workflow

Let's answer the following questions to start better understanding the `build.yml` workflow.

1. How many jobs are there?

2. How many steps are there?

3. What which steps are actions and which are commands

4. What is the type of runner

### Curated list of awesome actions to use on GitHub 🎉

- <https://github.com/sdras/awesome-actions>